In [ ]:
import json
import re
from difflib import SequenceMatcher
import yfinance as yf
import google.generativeai as genai

# Configure Google Generative AI
key = 'XXXX'  # Insert your Gemini API key here
genai.configure(api_key=key)

# Similarity calculation
def calculate_similarity(text1, text2):
    text1 = re.sub(r'[\^\w\s]', '', text1.lower())
    text2 = re.sub(r'[\^\w\s]', '', text2.lower())

    sequence_similarity = SequenceMatcher(None, text1, text2).ratio()
    words1 = set(text1.split())
    words2 = set(text2.split())
    word_overlap = len(words1.intersection(words2)) / max(len(words1), len(words2))

    return (sequence_similarity * 0.6) + (word_overlap * 0.4)

# Load knowledge base
def load_knowledge_base(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading knowledge base: {e}")
        return []

# Find best match
def find_best_match(user_input, knowledge_base):
    best_match = None
    highest_similarity = 0

    for entry in knowledge_base:
        about_similarity = calculate_similarity(user_input, entry['about_me'])
        context_similarity = calculate_similarity(user_input, entry['context'])
        total_similarity = (about_similarity * 0.7) + (context_similarity * 0.3)

        if total_similarity > highest_similarity:
            highest_similarity = total_similarity
            best_match = entry

    return best_match, highest_similarity

# Get stock price
def get_stock_price(stock_symbol):
    stock_symbol = stock_symbol.strip().upper()
    if not stock_symbol:
        return "Please provide a valid stock symbol."

    try:
        stock = yf.Ticker(stock_symbol)
        info = stock.info
        price = info.get('regularMarketPrice') or info.get('currentPrice')
        if price:
            return f"The current price of {stock_symbol} is ${price:.2f}."
        return f"Stock information for {stock_symbol} is not available at the moment."
    except Exception:
        return f"Unable to fetch stock price for {stock_symbol}. Please verify the symbol."

# Extract stock symbol
def extract_stock_symbol(user_input):
    patterns = [
        r"stock\s+price\s+(?:of\s+)?([A-Za-z.\-]+)",
        r"price\s+(?:of\s+)?([A-Za-z.\-]+)\s+stock",
        r"tell\s+me\s+([A-Za-z.\-]+)\s+stock\s+price",
        r"([A-Za-z.\-]+)\s+stock\s+price",
        r"stock\s+([A-Za-z.\-]+)"
    ]

    for pattern in patterns:
        match = re.search(pattern, user_input, re.IGNORECASE)
        if match:
            return match.group(1).upper()
    return None

# Handle Google Gemini API query
def query_gemini_api(user_input):
    try:
        model = genai.GenerativeModel('gemini-1.5-flash')
        response = model.generate_content(user_input)
        return response.text
    except Exception as e:
        return f"An error occurred while processing your request with Gemini: {e}"

# Basic question handling
def handle_basic_questions(user_input):
    question_patterns = {
        "save_money": [r"save\s+money", r"saving\s+tips", r"budget", r"reduce\s+expenses", r"cut\s+spending"],
        "investment_tips": [r"\binvest\b", r"investment", r"stock\s+market\b", r"portfolio", r"returns"],
        "debt_management": [r"debt", r"loan", r"credit", r"mortgage", r"repayment"]
    }

    responses = {
        "save_money": [
            "Here are some effective money-saving strategies:",
            "1. Track all expenses using a budgeting app or spreadsheet",
            "2. Set up automatic transfers to a separate savings account",
            "3. Follow the 50/30/20 rule: 50% needs, 30% wants, 20% savings",
            "4. Look for ways to reduce regular bills (phone, internet, subscriptions)"
        ],
        "investment_tips": [
            "Here are some key investment principles:",
            "1. Start with a diversified portfolio of low-cost index funds",
            "2. Consider your risk tolerance and investment timeline",
            "3. Regularly rebalance your portfolio",
            "4. Take advantage of tax-advantaged accounts when available"
        ],
        "debt_management": [
            "Here's a structured approach to managing debt:",
            "1. List all debts with their interest rates",
            "2. Use either the avalanche method (highest interest first) or snowball method (smallest balance first)",
            "3. Consider debt consolidation for high-interest debts",
            "4. Always pay more than the minimum payment when possible"
        ],
        "default": [
            "I can help you with:",
            "- Saving and budgeting strategies",
            "- Investment advice and portfolio management",
            "- Debt management and reduction",
            "What specific area would you like to learn more about?"
        ]
    }

    for category, patterns in question_patterns.items():
        if any(re.search(pattern, user_input.lower()) for pattern in patterns):
            return responses[category]

    return responses["default"]

# Main chatbot loop
def chatbot():
    knowledge_base = load_knowledge_base('/content/custom_data.json')
    print("\nHello! I'm your financial advice assistant. How can I help you today?")
    print("You can ask about:")
    print("- Saving money and budgeting")
    print("- Investment advice")
    print("- Debt management")
    print("- Stock prices (e.g., 'stock price AAPL')")
    print("Or share your financial situation for personalized advice.")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ["exit", "quit", "bye", "goodbye"]:
            print("Thank you for using the financial advisor chatbot. Have a great day!")
            break

        if not user_input:
            print("Please type your question or concern.")
            continue

        if knowledge_base:
            match, similarity = find_best_match(user_input, knowledge_base)
            if match and similarity > 0.3:
                print(f"Chatbot: {match['response']}")
                continue

        basic_responses = handle_basic_questions(user_input)
        if basic_responses != handle_basic_questions(""):
            for response in basic_responses:
                print(f"Chatbot: {response}")
            continue

        stock_symbol = extract_stock_symbol(user_input)
        if stock_symbol:
            response = get_stock_price(stock_symbol)
            print(f"Chatbot: {response}")
            continue

        gemini_response = query_gemini_api(user_input)
        print(f"Chatbot (via Gemini): {gemini_response}")

chatbot()



Hello! I'm your financial advice assistant. How can I help you today?
You can ask about:
- Saving money and budgeting
- Investment advice
- Debt management
- Stock prices (e.g., 'stock price AAPL')
Or share your financial situation for personalized advice.

You: I am a 26-year-old software developer living in Japan. I have been saving money and I am interested in investing in a thematic ETF.
Chatbot: Given your age and profession, a Roth IRA is an excellent choice. Contributing now with after-tax dollars allows your investments to grow tax-free, and you can withdraw tax-free in retirement. This is beneficial if you expect to be in a higher tax bracket later. Start with low-cost index funds or ETFs to diversify and minimize fees.

You: exit
Thank you for using the financial advisor chatbot. Have a great day!


In [ ]:
!pip install yfinance google-generativeai